# OCBA-m

Source. Chen et al. (2008) 

Efficient Simulation Budget Allocation for Selecting an Optimal Subset


https://pubsonline.informs.org/doi/pdf/10.1287/ijoc.1080.0268

In [1]:
import pandas as pd
import numpy as np
from numba import jit
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## Utility funcs

In [2]:
def load_system(file_name, system, reps, reps_available, delim=','):
    """
    Reads system data from a txt file (assumes comma delimited by default).  
    Assumes that each column represents a system.
    Returns a numpy array.  Each row is a system (single row); each col is a replication
    
    
    @file_name = name of file containing csv data
    @system = index of system in txt file.
    @reps = replications wanted
    @reps_available = total number of replications that has been simulated.
    @delim = delimiter of file.  Default = ',' for CSV.  

    """
    
    return np.genfromtxt(file_name, delimiter=delim, usecols = system, skip_footer = (reps_available - reps))

In [3]:
def simulate(data, k, allocations):
    """
    Simulates the systems.  
    Each system is allocated a different budget of replications
    
    Returns list of numpy arrays
    
    @allocations = numpy array.  budget of replications for the k systems 
    """
    return [data[i][0:allocations[i]] for i in range(k)]
    

In [4]:
def get_ranks(array):
    """
    Returns a numpy array containing ranks of numbers within a input numpy array
    e.g. [3, 2, 1] returns [2, 1, 0]
    e.g. [3, 2, 1, 4] return [2, 1, 0, 3]
        
    @array - numpy array (only tested with 1d)
        
    """
    temp = array.argsort()
    ranks = np.empty_like(temp)
    ranks[temp] = np.arange(len(array))
    return ranks

In [5]:
def bootstrap(data, boots=1000):
    """
    Returns a numpy array containing the bootstrap resamples
    Useful for creating a large number of experimental datasets for testing R&S routines
    
    @data = numpy.array of systems to boostrap
    @boots = number of bootstrap (default = 1000)
    """

    experiments = boots
    designs = data.shape[0]
    samples = data.shape[1]

    datasets = np.zeros((experiments, designs, samples))
     
    for exp in range(experiments):
        
        for design in range(designs):

            for i in range(samples):

                datasets[exp][design][i] = data[design][round(np.random.uniform(0, samples)-1)]
      
    return datasets

In [6]:
@jit(nopython=True)
def crn_bootstrap(data, boots=1000):
    """
    Returns a numpy array containing the bootstrap resamples
    Useful for creating a large number of experimental datasets for testing R&S routines
    
    @data = numpy.array of systems to boostrap
    @boots = number of bootstrap (default = 1000)
    """

    experiments = boots
    designs = data.shape[0]
    samples = data.shape[1]
    
    datasets = np.zeros((experiments, designs, samples))
     
    for exp in range(experiments):
        
         for i in range(samples):

                row = data.T[np.random.choice(data.shape[0])]
                
                for design in range(designs):
                    datasets[exp][design][i] = row[design]  
      
    return datasets

# Walk through of how Ocba-m works

##  Input

**k** = total number of designs

**m** = no. of top designs to select

**T** = total budget (replications)

**n_0** = l number of replications  (>=5)

**delta** - additional budget to allocate between each simulation  

*where T - k.n_0 is a multiple of delta*


In [7]:
k = 10
m = 3
T = 2000
n_0 = 20
delta = 50

#specific to this implementation
ifile_name = 'data/EG1a.csv'
reps_available = 10000

### OCBA-m tutorial - single loop of the algorithm

The code below illustrates the implementation of OCBA-m with a single pass of the algorithm.

## Initialise

In [8]:
allocations = np.full(k, n_0, dtype=int)
allocations

array([20, 20, 20, 20, 20, 20, 20, 20, 20, 20])

In [9]:
data = load_system(ifile_name)
systems = simulate(data, k, allocations)

TypeError: load_system() missing 3 required positional arguments: 'system', 'reps', and 'reps_available'

In [ ]:
#sample means, standard devs and standard errors
means = np.array([array.mean() for array in systems])

In [ ]:
stds = np.array([array.std() for array in systems])
ses = np.divide(stds, np.sqrt(allocations))

In [ ]:
means

In [ ]:
ses

In [ ]:
#calculate paramater c and gammas

order = np.argsort(means)
s_ses = ses[order]
s_means = means[order]

order

In [ ]:
s_means

In [ ]:
s_ses

In [ ]:
c = ((s_ses[k-m+1] * s_means[k-m]) + (s_ses[k-m] * s_means[k-m+1]))/(s_ses[k-m]+s_ses[k-m+1])  
deltas = means - c

calculate c 

In [ ]:
c

In [ ]:
deltas

## Allocate 

allocate additional budget **delta**

Procedure:

1. calculate ratio N_i/(se_i /gamma_i)^2

2. while additional budget delta is available

    2.1 rank

    2.2 add 1 to smallest 

    2.3 delta =- 1


In [ ]:
values = np.divide(allocations, np.square(np.divide(ses, deltas)))
values

In [ ]:

for i in range(delta):

    ranks = get_ranks(values)
    allocations[ranks.argmin()] += 1
    #recalculate values.
    values = np.divide(allocations, np.square(np.divide(ses, deltas)))

values

In [ ]:
allocations

## Simulate

Simulate the next stage...

In [ ]:
systems = simulate(data, k, allocations)
means2 = np.array([array.mean() for array in systems])
print(np.subtract(means2, means))


In [ ]:
means = means2

and continue looping until budget is all used up...

### Full Implementation 

In [10]:
@jit(nopython=True)
def summary_statistics(systems, allocations):
    means = np.array([array.mean() for array in systems])
    stds = np.array([array.std() for array in systems])
    ses = np.divide(stds, np.sqrt(allocations))
    
    return means, ses

In [11]:
def top_m(means, m):
    """
    Returns the top m system designs
    Largest sample means
    """
    ranks = get_ranks(means)
    return np.argpartition(ranks, -m)[-m:]

In [12]:
def bottom_m(means, m):
    """
    Returns the bottom m system designs
    Smallest sample means
    """
    ranks = get_ranks(means)
    return np.argpartition(ranks, m)[:m]

In [13]:
@jit(nopython=True)
def parameter_c(means, ses, k, m):
    order = np.argsort(means)
    s_ses = ses[order]
    s_means = means[order]

    return((s_ses[k-m+1] * s_means[k-m]) + (s_ses[k-m] * s_means[k-m+1]))/(s_ses[k-m]+s_ses[k-m+1])  

In [38]:
def ocba_m(dataset, k, allocations, T, delta, m):
    
    while allocations.sum() < T:
        
        #simulate systems using new allocation of budget
        reps = simulate(dataset, k, allocations) 
        
        #calculate sample means and standard errors
        means, ses = summary_statistics(reps, allocations)
        
        #calculate parameter c and deltas
        c = parameter_c(means, ses, k, m)
        deltas = means - c
        
        print(deltas)
        
        #allocate
        for i in range(delta):
            values = np.divide(allocations, np.square(np.divide(ses, deltas)))
            ranks = get_ranks(values)
            allocations[ranks.argmin()] += 1
            
    return means, ses, allocations

In [15]:
def cs(selected_top_m, true_top_m):
    """
    Returns boolean value.  
    True = correct selection of top m
    False = incorrect selection (one or more of selected top m is incorrect)
    
    Keyword arguments:
    selected_top_m - numpy.array containing the indexes of the top m means selected by the algorithm
    true_top_m - numpy.array containing the indexes of the true top m means
    
    """
    return np.array_equal(np.sort(selected_top_m), true_top_m)

In [16]:
def oc(true_means, selected_top_m, true_top_m):
    """
    Return the opportunity cost of the selection
    Method penalised particular bad solutions more than mildly bad
    
    @true_means - numpy.array of true means
    @selected_top_m - numpy.array containing the indexes of the top m means selected by the algorithm
    @true_top_m - numpy.array containing the indexes of the true top m means
    
    """
    return (true_means[selected_top_m] - true_means[true_top_m]).sum()

In [17]:
def p_cs(correct_selections):
    """
    Returns the probability of correct selection P{cs}
    Keyword arguments:
    
    correct_selections - list indicating if a given budget found the top m e.g. [False, True, True]
    """
    return np.array(correct_selections).sum() / len(correct_selections)

In [18]:
def p_cs2(correct_selections):
    """
    Returns the probability of correct selection P{cs}
    
    @correct_selections - numpy.array of budget versus exp. Contains True or False indicating correct selection.
    
    """
    return np.mean(correct_selections, axis=1)

In [19]:
def e_oc(opportunity_costs):
    """
    Return the expected opportunity cost E{oc}
    
    @opportunity costs - arrange of opportunity cost per budget
    """
    return np.array(opportunity_costs).mean()

In [20]:
def e_oc2(opportunity_costs):
    """
    Return the expected opportunity cost E{oc}
    
    @opportunity costs - arrange of opportunity cost per budget
    """
    return np.mean(opportunity_costs, axis=0)

# Numerical test - single experiment

### Create Experimental Data#

#### Use the following code to create independent samples

In [21]:
def experiments_independent_samples(ifile_name, boots=1000):
    data = np.genfromtxt(ifile_name, delimiter=",", skip_footer=0).transpose()
    experiments = bootstrap(data, boots=boots)
    return experiments

#### Use the following code to create CRN data sets

In [ ]:
def experiments_dependent_samples(ifile_name, boots=1000):
    data = np.genfromtxt(ifile_name, delimiter=",", skip_footer=0).transpose()
    experiments = crn_bootstrap(data, boots=boots)
    return experiments

Use the `numerical_experiment()` function to run experiments of OBCA-m across multiple experimental datasets and budgets.

For example, you may have 100 datasets to test across budgets of 1000, 2000, 3000 and 4000 replications.

In [37]:
def numerical_experiment(experiments, budgets, delta, true_top_m, n_0,
                         opt='max'):
    """
    Conduct a user set number of numerical experiments on the algorithm
    for different computational budgets
    
    Returns:
    1. numpy.array containing P{cs} for each budget
    2. numpy.array containing E{oc} for each budget
    
    Keyword arguments:
    experiments -- numpy.array[experiments][designs][replication]
    budgets -- python.list containing budgets
    delta - maximum amount of budget to allocate per run. 
    true_top_m -- top m (i.e. the systems that should be returned)
    n_0 -- initial number of replications per system
    
    """
    n_experiments = experiments.shape[0]
    k = experiments.shape[1]
    m = true_top_m.shape[0]
        
    correct_selections = np.zeros((n_experiments, len(budgets)))
    opportunity_costs = np.zeros((n_experiments, len(budgets)))

    for exp in range(n_experiments):

        for t in range(len(budgets)):

            allocations = np.full(k, n_0, dtype=int)
                        
            means, ses, allocations = ocba_m(experiments[exp], k, 
                                             allocations, budgets[t], 
                                             delta, m)
            
            #print(allocations)
            #print(ses)
            #print(means)
            
            if opt == 'max':
                selected_top_m = top_m(means, m)
                print('max')
            else:
                selected_top_m= bottom_m(means, m)

            correct_selections[exp][t] = cs(selected_top_m, true_top_m)
            print(selected_top_m)
            
    return correct_selections

# Experiments for CRN data

In [24]:
def get_budgets(max_t, min_t, increment_t):
    #incremental budgets 200, 400, .... T
    budgets = [i for i in range(min_t, max_t + increment_t, increment_t)]
    return budgets

In [28]:
def experiment_1():
    T = 2000
    increment_t = 100
    min_budget = 300
    n_0 = 20
    delta = 50

    #data file for this experiment.
    ifile_name = 'data/EG1a_CRN.csv'

    #info for correct selection
    true_top_m = np.array([7, 8, 9])

    #incremental budgets 200, 400, .... T
    budgets = [400] #get_budgets(T, min_budget, increment_t)
    
    #generate experimental dataset
    experiments = experiments_independent_samples(ifile_name, boots=10)
    
    #run numerical experiment
    css = numerical_experiment(experiments, budgets, delta, true_top_m, n_0)
    
    return css

In [39]:
css = experiment_1()

[-8.15468628 -6.33116906 -4.25496896 -6.02227429 -3.28087673 -4.92773213
  0.98581728 -1.52597796 -0.42311312  0.36858027]
[-8.19123787 -6.36772065 -4.29152055 -6.05882589 -3.31742832 -4.96428372
  0.94926569 -1.56252955 -0.61185695  0.44094849]
[-8.36022679 -6.53670957 -4.46050946 -6.2278148  -3.48641724 -5.13327263
  0.55596527 -1.73151846 -0.55249048  0.50070007]
[-7.637705   -5.81418778 -3.73798768 -5.50529302 -2.76389545 -4.41075085
 -0.13381105 -1.00899668  0.1700313   1.22322185]
max
[6 8 9]
[-7.05999847 -6.02833571 -4.94769424 -4.68923562 -1.50273314 -5.14351885
 -2.56468914 -1.96466674  1.300477    1.20082871]
[-6.54861702 -5.51695427 -4.4363128  -4.17785417 -1.89198389 -4.6321374
 -2.05330769 -1.28713551  1.69859373  0.8472416 ]
[-6.64850948 -5.61684673 -4.53620526 -4.27774664 -1.99187635 -4.73202986
 -1.64286779 -1.4901837   1.59870126  1.71789135]
[-5.91856062 -4.88689787 -3.8062564  -3.54779777 -1.26192749 -4.002081
 -1.44526317 -0.76023484  0.72504333  2.44784021]
max
[7 

In [30]:
df = pd.DataFrame(css)
df.to_clipboard()

In [31]:
df

,0
0,1.0
1,1.0
2,1.0
3,1.0
4,1.0
5,1.0
6,0.0
7,1.0
8,1.0
9,0.0


## Experiments for Law Inventory Example

### Maximisation Example (Top 3)

In [ ]:
def experiment_2():
    T = 300
    increment_t = 100
    min_budget = 300
    n_0 = 20
    delta = 50
    n_experiments = 5

    #data file for this experiment
    ifile_name = 'data/EGLaw_CRN.csv'
    
    #info for correct selection (maximisation problem)
    true_top_m = np.array([6, 7, 8]) 
    
    #incremental budgets 200, 400, .... T
    budgets = get_budgets(T, min_budget, increment_t)
    
    #generate experimental dataset
    experiments = experiments_dependent_samples(ifile_name, boots=n_experiments)
    
    #run numerical experiment
    css = numerical_experiment(experiments, budgets, delta, true_top_m, n_0)
    
    return css

### Minimisation Example (Bottom 2)

In [ ]:
def experiment_3():
    T = 1000
    increment_t = 100
    min_budget = 300
    n_0 = 20
    delta = 50
    n_experiments = 10000

    #data file for this experiment
    ifile_name = 'data/EGLaw_CRN.csv'
    
    #info for correct selection (maximisation problem)
    true_top_m = np.array([1, 2]) 
    
    #incremental budgets 200, 400, .... T
    budgets = get_budgets(T, min_budget, increment_t)
    
    #generate experimental dataset
    experiments = experiments_dependent_samples(ifile_name, boots=n_experiments)
    
    #run numerical experiment
    css = numerical_experiment(experiments, budgets, delta, true_top_m, n_0, 
                               opt='min')
    
    return css

# Run Experiments

In [ ]:
results_law_max = experiment_2()

In [ ]:
pd.DataFrame(results_law_max)

In [ ]:
results_law_min = experiment_3()

In [ ]:
pd.DataFrame(results_law_min)

In [ ]:
#df = pd.DataFrame(css)
#df["select"] = top_m_list
#df["allocations"] = allocation_list
#df["means_list"] = np.around(means_list, decimals=4).tolist()
#df.head()

In [ ]:
#print("top m: {0}".format(df[0:1]["select"].tolist()))
#print("Means selected: {0}".format(df[0:1]["means_list"].tolist()))
#print("Allocations: {0}".format(df[0:1]["allocations"].tolist()))

In [ ]:
budgets = get_budgets(3000, 300, 100)
fig = plt.figure(figsize=(10,6))
ax = pd.DataFrame(css, columns=budgets).mean(axis=0).plot(style='.-', ms=15)
ax.set_ylabel("P{CS}")
ax.set_xlabel("Budget (max no. replications)")
ax.set_xticks(ticks = [i for i in range(200, 8000, 200)])

ax.plot()

In [ ]:
fig = plt.figure(figsize=(10,6))
ax = pd.DataFrame(ocs, columns=budgets).mean(axis=0).plot(style='.-', ms=15)
ax.set_ylabel("E{OC}")
ax.set_xlabel("Budget (max no. replications)")
ax.set_xticks(ticks = [i for i in range(200, 8000, 200)])

ax.plot()